# import 

In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder


# load the dataset

In [20]:
df = pd.read_csv('dataset.csv')
df = df[['Date' , 'HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']]


In [21]:
df= pd.get_dummies(df, columns=['FTR','HTR'])

In [22]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,...,HY,AY,HR,AR,FTR_A,FTR_D,FTR_H,HTR_A,HTR_D,HTR_H
0,2024-09-01,Kortrijk,St Truiden,1.0,1.0,0.0,1.0,15.0,8.0,4.0,...,1.0,1.0,0.0,0.0,False,True,False,True,False,False
1,2024-09-01,St. Gilloise,Anderlecht,0.0,0.0,0.0,0.0,22.0,10.0,6.0,...,3.0,4.0,0.0,0.0,False,True,False,False,True,False
2,2024-09-01,Gent,Antwerp,1.0,1.0,1.0,1.0,15.0,15.0,8.0,...,4.0,3.0,0.0,0.0,False,True,False,False,True,False
3,2024-09-01,Club Brugge,Cercle Brugge,3.0,0.0,2.0,0.0,21.0,11.0,10.0,...,1.0,1.0,0.0,0.0,False,False,True,False,False,True
4,2024-08-31,Oud-Heverlee Leuven,Standard,2.0,0.0,1.0,0.0,13.0,4.0,4.0,...,4.0,4.0,0.0,0.0,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2019-07-27,Waasland-Beveren,Club Brugge,1.0,3.0,1.0,1.0,7.0,25.0,2.0,...,3.0,1.0,0.0,0.0,True,False,False,False,True,False
1504,2019-07-27,St Truiden,Mouscron,0.0,1.0,0.0,1.0,10.0,10.0,4.0,...,2.0,4.0,0.0,0.0,True,False,False,True,False,False
1505,2019-07-27,Waregem,Mechelen,0.0,2.0,0.0,1.0,7.0,10.0,2.0,...,3.0,1.0,0.0,0.0,True,False,False,True,False,False
1506,2019-07-27,Cercle Brugge,Standard,0.0,2.0,0.0,0.0,13.0,14.0,5.0,...,2.0,2.0,1.0,0.0,True,False,False,False,True,False


In [23]:
matches = df.groupby(['HomeTeam','AwayTeam'])[['FTR_A','FTR_D','FTR_H','HTR_A','HTR_D','HTR_H']].sum().reset_index()
matches_stats = df.groupby(['HomeTeam','AwayTeam'])[['FTHG','FTAG','HTHG','HTAG','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']].mean().reset_index()

matches = pd.merge(matches, matches_stats, on=['HomeTeam','AwayTeam'])
matches['total_matches'] = matches['FTR_A'] + matches['FTR_D'] + matches['FTR_H']
matches.value_counts('total_matches')

total_matches
2    141
5     85
4     78
6     51
1     49
3     33
7      5
Name: count, dtype: int64

In [25]:
model = RandomForestRegressor(random_state=42)
encoder = OrdinalEncoder()
X = matches.drop(columns=['HomeTeam','AwayTeam'])
y = matches[['HomeTeam','AwayTeam']]
encoder.fit(y)
y_encoder = encoder.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)


       HomeTeam       AwayTeam
0    Anderlecht        Antwerp
1    Anderlecht   Beerschot VA
2    Anderlecht  Cercle Brugge
3    Anderlecht      Charleroi
4    Anderlecht    Club Brugge
..          ...            ...
437    Westerlo        Seraing
438    Westerlo     St Truiden
439    Westerlo   St. Gilloise
440    Westerlo       Standard
441    Westerlo        Waregem

[442 rows x 2 columns]


ValueError: could not convert string to float: 'Anderlecht'

In [27]:
teams = matches[['HomeTeam','AwayTeam']]
print(teams)


       HomeTeam       AwayTeam
0    Anderlecht        Antwerp
1    Anderlecht   Beerschot VA
2    Anderlecht  Cercle Brugge
3    Anderlecht      Charleroi
4    Anderlecht    Club Brugge
..          ...            ...
437    Westerlo        Seraing
438    Westerlo     St Truiden
439    Westerlo   St. Gilloise
440    Westerlo       Standard
441    Westerlo        Waregem

[442 rows x 2 columns]
